# ПОСТАНОВКА ЗАДАЧИ
В прошлых блоках мы группировали датафреймы по столбцам и получали набор метрик для них. Давайте расширим нашу задачу - требуется получить для каждого пользователя распределение по количеству выставленных оценок. Т. е. в строках таблицы указывается ID пользователя, а в столбцах - количество выставленных рейтингов: 0.5, 1.0, ..., 5.0.

Для получения такого расширенного варианта группировок применяются сводные таблицы. Это те же сводные таблицы, которые широко используются в Excel.

# ПРОСТЫЕ СВОДНЫЕ ТАБЛИЦЫ
Начинаем решение задачи с привычной загрузки данных о рейтингах:

In [8]:
import pandas as pd


data = pd.read_csv('./module05_files/ratings.csv')
pd.set_option('display.max_rows', 10)

Если мы хотим получить распределение количества оценок для одного заданного пользователя, то можно было бы просто использовать метод value_counts. Например, для пользователя 1:

In [5]:
data[ data['userId'] == 1 ]['rating'].value_counts()

2.0    7
3.0    4
4.0    3
2.5    3
1.0    2
3.5    1
Name: rating, dtype: int64

Чтобы получить такую таблицу для всех пользователей давайте в строки поставим значения userId, а в столбцы - значения рейтинга rating. В ячейки поместим количество строк. В итоге получим классическую сводную таблицу:

In [9]:
data.pivot_table(index='userId', 
                 columns='rating', 
                 values='timestamp', 
                 aggfunc='count')

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,NaN,2.0,NaN,7.0,3.0,4.0,1.0,3.0,NaN,NaN
2,NaN,2.0,NaN,4.0,NaN,36.0,NaN,23.0,NaN,11.0
3,NaN,NaN,NaN,1.0,3.0,18.0,9.0,11.0,4.0,5.0
4,NaN,5.0,NaN,5.0,NaN,23.0,NaN,52.0,NaN,119.0
5,NaN,NaN,1.0,NaN,3.0,3.0,27.0,42.0,19.0,5.0
...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,4.0,NaN,28.0,NaN,24.0,NaN,12.0
668,NaN,2.0,NaN,NaN,NaN,7.0,NaN,3.0,NaN,8.0
669,NaN,NaN,NaN,7.0,NaN,14.0,NaN,12.0,NaN,4.0


# НЕОПРЕДЕЛЕННЫЕ ЗНАЧЕНИЯ
В нашей таблице довольно много неопределенных значений (ячеек с NaN). Довольно логично, т. к. далеко не все пользователи выставляют все значения от 0.5 до 5.0. Чтобы избавиться от них, добавим в сводную таблицу параметр  fill_value. И заменим эти значения на 0:

In [10]:
data_pivot = data.pivot_table(index = 'userId', 
                              columns = 'rating', 
                              values = 'timestamp', 
                              aggfunc = 'count', 
                              fill_value = 0)

data_pivot.head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0
2,0,2,0,4,0,36,0,23,0,11
3,0,0,0,1,3,18,9,11,4,5
4,0,5,0,5,0,23,0,52,0,119
5,0,0,1,0,3,3,27,42,19,5


Если в сводной таблице необходимо добавить столбец итогов (All), то используем еще один дополнительный параметр margins=True:

In [11]:
data_pivot = data.pivot_table(index = 'userId', 
                              columns = 'rating', 
                              values = 'timestamp', 
                              aggfunc = 'count', 
                              fill_value = 0, 
                              margins=True)

data_pivot.head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0,20
2,0,2,0,4,0,36,0,23,0,11,76
3,0,0,0,1,3,18,9,11,4,5,51
4,0,5,0,5,0,23,0,52,0,119,204
5,0,0,1,0,3,3,27,42,19,5,100


# Проверочное задание
(1 возможный балл)
Постройте рейтинг пользователей по количеству выставленных оценок 5.0.

Какое максимальное количество 'пятерок' выставил один пользователь?

In [16]:
data_pivot[5].sort_values()

userId
1          0
77         0
79         0
591        0
579        0
       ...  
547      214
242      219
232      243
564      408
All    15095
Name: 5.0, Length: 672, dtype: int64